## W 홈페이지에서 특정 브랜드 검색 후, 기사 제목 & 상세 페이지 주소 & 기사 작성일 크롤링
- http://www.wkorea.com/

In [3]:
import pandas as pd

In [4]:
from selenium import webdriver
from selenium.webdriver.common.by import By   # 특정 요소 존재 여부 확인
from selenium.webdriver.support.ui import WebDriverWait # 웹 페이지 로딩 완료될 때까지 대기
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys # key event 발생 시킴

# Exception
from selenium.common.exceptions import TimeoutException, NoSuchElementException

In [5]:
path = '../driver/chromedriver.exe'
wUrl = 'http://www.wkorea.com/?s='

In [6]:
def makeUrl(pUrl, pSearchWord) :            
    return str(pUrl + pSearchWord)

In [7]:
searhWord = {'셀린' : 'Ce',
             '디올'   : 'D',
             '샤넬'   : 'Ch',
             '로랑' : 'YSL'} # 입생로랑

In [8]:
searchWordIndex = 0

In [9]:
brand = list(searhWord.keys())[searchWordIndex]
brand

'셀린'

In [10]:
engBrand = list(searhWord.values())[searchWordIndex]
engBrand

'Ce'

In [11]:
Url = wUrl
driver = webdriver.Chrome(path)     # 크롬 브라우저 제어할 수 있는 객체, 웹브라우저 실행
driver
driver.get(makeUrl(Url, list(searhWord.keys())[searchWordIndex]))

### 검색한 페이지

In [7]:
# title로 상세 페이지 주소
a_title = driver.find_element_by_css_selector('.category-txt-box > div > h2 > a').get_attribute('href')
print(a_title)

http://www.wkorea.com/2020/09/22/%ec%85%80%eb%a6%b0%eb%8a%90-%ea%b8%80%eb%a1%9c%eb%b2%8c-%ec%97%a0%eb%b2%84%ec%84%9c%eb%8d%94%ea%b0%80-%eb%90%9c-%eb%b8%94%eb%9e%99%ed%95%91%ed%81%ac-%eb%a6%ac%ec%82%ac/


In [8]:
# img로 상세 페이지 주소
a_img = driver.find_element_by_css_selector('.slides > li > div > a').get_attribute('href')
print(a_img)

http://www.wkorea.com/2020/09/22/%ec%85%80%eb%a6%b0%eb%8a%90-%ea%b8%80%eb%a1%9c%eb%b2%8c-%ec%97%a0%eb%b2%84%ec%84%9c%eb%8d%94%ea%b0%80-%eb%90%9c-%eb%b8%94%eb%9e%99%ed%95%91%ed%81%ac-%eb%a6%ac%ec%82%ac/


In [9]:
# title 
title = driver.find_element_by_css_selector('.category-txt-box > div > h2 > a').text
title

'셀린느 글로벌 엠버서더가 된 블랙핑크 리사'

In [10]:
# 화보, 이미지
img = driver.find_element_by_css_selector('.slides > li > div > a > img').get_attribute('src')
img

'http://img.wkorea.com/w/2020/09/style_5f69c8611e753-500x354-1600768312.jpg'

In [25]:
# 카테고리
category = driver.find_element_by_css_selector('.category-txt-box > div.fusion-meta-info > div').text[11:]
category

"WHAT'S NEW"

In [72]:
# 다음 페이지 click event
btnNextPage = driver.find_element_by_css_selector('.pagination-next')
btnNextPage.click()

In [35]:
# 해당 태그 존재 여부 확인
# 해당 태그가 없을 경우에는 빈 리스트
btnNext = driver.find_elements_by_css_selector('.pagination-next')
print(btnNext)

if len(btnNext) > 0 :
    print('True')
else :
    print('False')     

[<selenium.webdriver.remote.webelement.WebElement (session="d0461201dc0cefef9c44f1fd70fdce5b", element="c2ccf471-de9d-4e34-ae9b-80a6504ee520")>]
True


In [12]:
# 다음 페이지 이동 버튼 존재 여부
def existenceBtnNextPage(pBtnNext) :     
    print('driver.current_url :', driver.current_url)    
    result = False
    
    try : 
        result = True if len(pBtnNext) > 0 else False
        
    except Exception as e :
        result = False
        
    finally :
        return result

In [13]:
# 원하는 요소 정보 추출
def getElement() :

    elements = driver.find_elements_by_css_selector('#posts-container > div > article')
    # print(elements)

    for element in elements :
        
        category = element.find_element_by_css_selector('.category-txt-box > div.fusion-meta-info > div').text[11:]
        categoryList.append(category)
        
        title = element.find_element_by_css_selector('.category-txt-box > div > h2 > a').text
        titleList.append(title)

        a_href = element.find_element_by_css_selector('.category-txt-box > div > h2 > a').get_attribute('href')
        linkList.append(a_href)

        img = element.find_element_by_css_selector('.slides > li > div > a > img').get_attribute('src')
        imgList.append(img)

        # yyyy.MM.dd
        date = element.find_elements_by_css_selector('.category-txt-box > div')[2].text[:10]
        dateList.append(date)

In [14]:
btnNextPage = driver.find_elements_by_css_selector('.pagination-next')

In [15]:
# 다음 페이지 이동 버튼 존재 여부 확인 및 다음 페이지 이동
def getSearchResult(pCssSelector) :    
    
    delay = 10 # 로딩 완료 대기 시간(초 단위)
    pageNum = 0 # debug용

    while True:    
        pageNum+=1
        print('page : ', pageNum)
            
        try :  
            
            getElement()
            
            # element 대입 시, 다음 페이지 이동 버튼(pCssSelector)이 없을 경우 찾는데 시간이 많이 소요되서 TimeoutException 발생
            element = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.CSS_SELECTOR, pCssSelector)))
            
            # 다음 페이지 이동 버튼 있을 경우 Click Event 
            # Click Event 발생시, 마지막 페이지 이동되기 전에 오류 발생하는 경우 있어서 key event로 변경
            element.send_keys(Keys.ENTER)
        
        except TimeoutException as toe:
            
            btnNext = driver.find_elements_by_css_selector(pCssSelector)
            
            # 해당 css selector가 없어서 못 읽어온 건지, 있는데 읽는데 오래 걸려서인지 확인
            print('TimeoutException : ', toe)  if existenceBtnNextPage(btnNext) else print('NoSuchElementException')            
            return   

        except Exception as e :
            print('예외 발생 :', e)
            return 

In [16]:
cssSelector = '.pagination-next'

In [17]:
# list 초기화
categoryList = []
titleList = []
linkList = []
imgList = []
dateList = []
    
getSearchResult(cssSelector)

page :  1
page :  2
page :  3
page :  4
page :  5
page :  6
page :  7
page :  8
page :  9
page :  10
page :  11
page :  12
page :  13
page :  14
page :  15
page :  16
page :  17
page :  18
page :  19
page :  20
page :  21
page :  22
page :  23
page :  24
page :  25
page :  26
page :  27
page :  28
page :  29
page :  30
page :  31
page :  32
page :  33
page :  34
page :  35
page :  36
page :  37
page :  38
page :  39
page :  40
page :  41
page :  42
page :  43
page :  44
page :  45
page :  46
driver.current_url : http://www.wkorea.com/page/46/?s=%EC%85%80%EB%A6%B0
NoSuchElementException


In [50]:
print('len :', len(categoryList))
print(categoryList)

len : 694
['FASHION, 뉴스', 'BEAUTY, 뉴스', 'BEAUTY, 뉴스', 'BEAUTY, 뉴스', 'BEAUTY, 뉴스', 'FASHION, 뉴스', 'FASHION, 뉴스', 'FASHION, 뉴스', '뉴스', '뉴스', '라이프', 'FASHION, 쇼핑, 트렌드', 'FASHION, 쇼핑, 화보', 'FASHION, 쇼핑, 트렌드', 'FASHION, 트렌드, 화보', 'FEATURE, 피플', 'FASHION, 화보', 'FASHION, 화보', 'FASHION, 쇼핑, 트렌드', 'FASHION, 트렌드', 'FASHION, 트렌드', 'FEATURE, 피플', 'FEATURE, 컬처', 'BEAUTY, 화보', 'FASHION', 'FASHION, W맨, 쇼핑, 트렌드', 'FASHION, W맨, 쇼핑', 'FASHION, 화보', 'FASHION, W맨, 쇼핑, 트렌드', 'FASHION, 화보', 'FASHION, 쇼핑, 트렌드', 'FEATURE, 컬처', 'FASHION, 쇼핑, 트렌드', 'FASHION, 트렌드', 'BEAUTY, 쇼핑, 트렌드', 'FEATURE, 피플', 'FASHION, 쇼핑, 트렌드', 'W TV', 'FEATURE, 피플', 'FASHION, FEATURE, 피플, 화보', 'FASHION, 쇼핑, 트렌드', 'FASHION, 트렌드, 화보', 'BEAUTY, 쇼핑, 트렌드', 'FASHION, 뉴스', 'FASHION, FEATURE, 피플, 화보', 'FEATURE, 피플', 'FASHION, W맨, 트렌드', 'FASHION, 뉴스', 'FASHION, 트렌드', 'FASHION, 쇼핑, 트렌드', 'FASHION, 트렌드, 화보', 'FASHION, 쇼핑, 트렌드', 'FASHION, 쇼핑, 트렌드', 'FEATURE, 피플', 'FASHION, 트렌드', 'FASHION, 트렌드', 'FASHION, 트렌드', 'FASHION, 뉴스', 'FASHION, 화보', 'FASHION,

In [34]:
print('len :', len(titleList))
print(titleList)

len : 1121
['샤넬 홀리데이 컬렉션이 떴다!', '얼굴의 숨은 빛을 끌어올리는 샤넬 레베쥬', '샤넬 뷰티를 그리다', '샤넬 향수가 펜슬로 만들어졌다고?', '태양 빛을 머금은 피부, 샤넬의 레 베쥬 썸머 오브 글로우 컬렉션', '샤넬의 2020/21 크루즈 컬렉션 ‘지중해에서의 산책’', '배우 이동욱의 애정템! 향이 오래 지속되는 남자 향수 ‘블루 드 샤넬’', '#StayHappy 장윤주, 아이린, 김아현의 샤넬 코코 크러쉬', '이동욱의 동안 피부 비결은? ‘샤넬의 남자’ 이동욱 뷰티 루틴 알아보기', '샤넬 코드 코코, 코코 크러쉬의 찬란한 가을날', '샤넬을 사로잡은 트렌스젠더 모델', '샤넬의 세 여자', '샤넬 2020 Resort – Chanel', '퍼렐과 샤넬', '퍼렐표 샤넬 공개임박', '요트가 손목 위로, 샤넬의 뉴워치 THE J12', '빌라 샤넬', '샤넬 화인 주얼리, 코코 크러쉬 코코 랩(Coco Lab)이벤트!', '샤넬 2019 크루즈 컬렉션에 참석한 김고은', '샤넬 3.55 핸드백 스토리', '톱 스타일리스트들의 샤넬백 스타일링 6', '2018 S/S BEAUTY BOOK – 샤넬', 'Happy Boots Day! 경쾌한 샤넬 부츠 Film', '경쾌한 샤넬 부츠', '샤넬의 새로운 시계 코드 코코', '가브리엘 샤넬의 특별한 항해', '샤넬 2018 크루즈 메이크업', '샤넬 17/18 크루즈 컬렉션', '샤넬 #코코카페 #COCOCAFE', '샤넬 뷰티 토크 4번째 뮤즈 릴리-로즈 뎁', '#W뷰테나 – 샤넬 쿠튀르 메이크업', '샤넬 쇼에 등장한 금수저들', '샤넬의 강렬한 여름 색채', '샤넬 ‘파리 인 로마’ 크리스틴 스튜어트 필름', '샤넬이 스며든 쿠바의 숙소', '#16SS 샤넬 오트쿠튀르', '샤넬 ‘원스 앤 포에버(Once and Forever)’ 필름', '크리스틴 스튜어트의 샤넬 필름 ‘Once and forever pictures’', '샤넬의 전시 Mademoiselle Privé의

In [35]:
print('len :', len(linkList))
print(linkList)

len : 1121
['http://www.wkorea.com/2020/11/02/%ec%83%a4%eb%84%ac-%ed%99%80%eb%a6%ac%eb%8d%b0%ec%9d%b4-%ec%bb%ac%eb%a0%89%ec%85%98%ec%9d%b4-%eb%96%b4%eb%8b%a4/', 'http://www.wkorea.com/2020/10/29/%ec%96%bc%ea%b5%b4%ec%9d%98-%ec%88%a8%ec%9d%80-%eb%b9%9b%ec%9d%84-%eb%81%8c%ec%96%b4%ec%98%ac%eb%a6%ac%eb%8a%94-%ec%83%a4%eb%84%ac-%eb%a0%88%eb%b2%a0%ec%a5%ac/', 'http://www.wkorea.com/2020/08/12/%ec%83%a4%eb%84%ac-%eb%b7%b0%ed%8b%b0%eb%a5%bc-%ea%b7%b8%eb%a6%ac%eb%8b%a4/', 'http://www.wkorea.com/2020/08/04/%ec%83%a4%eb%84%ac-%ed%96%a5%ec%88%98%ea%b0%80-%ed%8e%9c%ec%8a%ac%eb%a1%9c-%eb%a7%8c%eb%93%a4%ec%96%b4%ec%a1%8c%eb%8b%a4%ea%b3%a0/', 'http://www.wkorea.com/2020/07/12/%ed%83%9c%ec%96%91-%eb%b9%9b%ec%9d%84-%eb%a8%b8%ea%b8%88%ec%9d%80-%ed%94%bc%eb%b6%80-%ec%83%a4%eb%84%ac%ec%9d%98-%eb%a0%88-%eb%b2%a0%ec%a5%ac-%ec%8d%b8%eb%a8%b8-%ec%98%a4%eb%b8%8c-%ea%b8%80/', 'http://www.wkorea.com/2020/06/10/%ec%83%a4%eb%84%ac%ec%9d%98-2020-21-%ed%81%ac%eb%a3%a8%ec%a6%88-%ec%bb%ac%eb%a0%89%ec%85%98-%ec%a7%80%e

In [36]:
print('len :', len(imgList))
print(imgList)

len : 1121
['http://img.wkorea.com/w/2020/11/style_5f9f9714243d7.gif', 'http://img.wkorea.com/w/2020/10/style_5f9a6068be141-500x354.jpg', 'http://img.wkorea.com/w/2020/08/style_5f3272ffa3f0e-500x354-1597142404.jpg', 'http://img.wkorea.com/w/2020/08/style_5f293cb056c15-500x354-1596538135.jpg', 'data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7', 'data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7', 'data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7', 'data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7', 'data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7', 'data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7', 'http://img.wkorea.com/w/2019/08/style_5d64e7ea4386f-500x354-1566894116.jpg', 'http://img.wkorea.com/w/2019/06/style_5cf77e3899ab2-500x354.jpg', 'http://img.wkorea.com/w/2019/05/style_5cd17cef9943d.gif', 'http://im

In [37]:
print('len :', len(dateList))
print(dateList)

len : 1121
['2020.11.02', '2020.10.29', '2020.08.12', '2020.08.04', '2020.07.12', '2020.06.10', '2020.05.27', '2020.05.25', '2020.05.22', '2019.11.06', '2019.08.27', '2019.06.07', '2019.05.07', '2019.03.27', '2019.03.25', '2019.03.25', '2019.02.28', '2018.11.23', '2018.05.24', '2018.04.20', '2018.03.22', '2018.02.21', '2017.11.30', '2017.11.30', '2017.11.21', '2017.08.14', '2017.05.15', '2017.05.04', '2017.03.20', '2017.03.14', '2017.02.03', '2016.12.09', '2016.10.25', '2016.06.17', '2016.05.13', '2016.01.26', '2015.12.08', '2015.11.20', '2015.10.14', '2015.08.05', '2015.07.16', '2015.07.10', '2015.07.07', '2015.05.10', '2015.05.07', '2015.01.23', '2014.09.01', '2014.08.29', '2014.08.13', '2014.06.15', '2014.05.22', '2014.05.21', '2014.05.15', '2014.02.19', '2013.12.12', '2013.01.28', '2012.09.03', '2020.11.07', '2020.11.06', '2020.11.03', '2020.11.03', '2020.11.01', '2020.10.23', '2020.10.17', '2020.10.16', '2020.10.15', '2020.10.12', '2020.10.11', '2020.10.09', '2020.10.07', '2020.10

In [18]:
# 포맷 변경
dateList = [date.replace('.', '-') for date in dateList]
dateList

['2020-09-22',
 '2020-04-16',
 '2017-04-03',
 '2020-11-24',
 '2020-11-09',
 '2020-11-08',
 '2020-11-07',
 '2020-11-06',
 '2020-11-02',
 '2020-10-27',
 '2020-10-26',
 '2020-10-15',
 '2020-10-15',
 '2020-10-11',
 '2020-10-07',
 '2020-10-06',
 '2020-10-02',
 '2020-09-29',
 '2020-09-29',
 '2020-09-08',
 '2020-09-08',
 '2020-09-06',
 '2020-09-04',
 '2020-09-01',
 '2020-08-27',
 '2020-08-26',
 '2020-08-24',
 '2020-08-23',
 '2020-08-05',
 '2020-07-07',
 '2020-06-29',
 '2020-06-17',
 '2020-06-06',
 '2020-06-02',
 '2020-05-11',
 '2020-05-08',
 '2020-04-29',
 '2020-04-28',
 '2020-04-20',
 '2020-04-11',
 '2020-04-10',
 '2020-03-25',
 '2020-03-12',
 '2020-03-08',
 '2020-03-06',
 '2020-03-05',
 '2020-03-04',
 '2020-03-02',
 '2020-03-01',
 '2020-02-25',
 '2020-02-24',
 '2020-02-22',
 '2020-02-05',
 '2020-01-22',
 '2019-12-31',
 '2019-12-22',
 '2019-12-06',
 '2019-12-05',
 '2019-12-01',
 '2019-11-30',
 '2019-11-28',
 '2019-11-25',
 '2019-11-25',
 '2019-11-14',
 '2019-10-27',
 '2019-10-12',
 '2019-10-

In [20]:
resDf = pd.DataFrame(data = {'brand' : engBrand,
                             'category' : categoryList,
                             'title' : titleList,
                             'date' : dateList,
                             'link' : linkList,
                             'title_brand' : 0,
                             'check' : 0,
                             'why' : ''})

In [21]:
resDf.head()

,brand,category,title,date,link,title_brand,check,why
0,Ce,WHAT'S NEW,셀린느 글로벌 엠버서더가 된 블랙핑크 리사,2020-09-22,http://www.wkorea.com/2020/09/22/%ec%85%80%eb%...,0,0,
1,Ce,WHAT'S NEW,셀린느 트리옹프 캔버스 백,2020-04-16,http://www.wkorea.com/2020/04/16/%ec%85%80%eb%...,0,0,
2,Ce,"FASHION, 뉴스",셀린느(Celine) 청담 플래그십 스토어 오픈,2017-04-03,http://www.wkorea.com/2017/04/03/%ec%85%80%eb%...,0,0,
3,Ce,"FASHION, 트렌드",We Love Vol.1,2020-11-24,http://www.wkorea.com/2020/11/24/we-love-vol-1/,0,0,
4,Ce,"FASHION, 쇼핑, 트렌드",아무도 없었다,2020-11-09,http://www.wkorea.com/2020/11/09/%ec%95%84%eb%...,0,0,


In [22]:
# 컬럼별 중복 개수 확인
len(resDf) - resDf.nunique()

brand          457
category       413
title            2
date            31
link             0
title_brand    457
check          457
why            457
dtype: int64

In [23]:
# 날짜 조건
resDf = resDf[resDf['date']>= '2016-09-00']

In [24]:
len(resDf)

220

In [25]:
# 크롤링 결과 csv로 만듬
resDf.to_csv('./data/w_'+ engBrand +'.csv', sep = '|', encoding = 'utf-8', index = False)

### 상세 페이지

In [81]:
# 부제 
subTitle = driver.find_element_by_css_selector('.post-content > div > p').text
subTitle

'공식적으로 에디 슬리먼 패밀리가 된 그녀!'

In [71]:
# 상세 페이지 기사
content = driver.find_element_by_css_selector('.post-content > div > p > p').text
content

'공식적으로 에디 슬리먼 패밀리가 된 그녀!'